In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import precision_score, f1_score, roc_auc_score, accuracy_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
df_short = pd.read_csv("data/processed/processed_short.csv")
df_medium = pd.read_csv("data/processed/processed_medium.csv")
df_dank = pd.read_csv("data/processed/processed_dank.csv")

df_joint = pd.concat([df_short, df_medium, df_dank])
df_joint = df_joint.reset_index(drop=True)
df_train = df_joint.sample(frac=1).reset_index(drop=True)

In [1]:
def model_trial(df):
    tfidf = TfidfVectorizer()
    name =[x for x in globals() if globals()[x] is df][0]
    #creating the desired vectors
    text = df['text']
    y = df["label"]
    text_train, text_test, y_train, y_test = train_test_split(
    text, y, test_size=0.33, random_state=42)
    X_train = tfidf.fit_transform(text_train.astype('U').values)
    X_test = tfidf.transform(text_test.astype('U').values)
    # D_train = xgb.DMatrix(X_train, label=y_train)
    # D_test = xgb. DMatrix(X_test, label=y_test)

    # #log
    # log = LogisticRegression()
    # log.fit(X_train, y_train)
    # y_pred_bool = log.predict(X_test)
    # accuracy = accuracy_score(y_test, y_pred_bool)
    # precision = precision_score(y_test, y_pred_bool)
    # recall = recall_score(y_test, y_pred_bool)
    # roc = roc_auc_score(y_test, y_pred_bool)
    
    
    # metrics = {'accuracy': accuracy, 'precision': precision, 'recall': recall, "roc": roc}
    # print('------------------Log-----------------------------')
    # print(metrics)
    # print('--------------------------------------------------')

    #svm
    svm = LinearSVC()
    svm.fit(X_train, y_train)
    y_pred_bool = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_bool)
    precision = precision_score(y_test, y_pred_bool)
    recall = recall_score(y_test, y_pred_bool)
    roc = roc_auc_score(y_test, y_pred_bool)
    
    
    # metrics = {'accuracy': accuracy, 'precision': precision, 'recall': recall, "roc": roc}
    # print('------------------SVM-----------------------------')
    # print(metrics)
    # print('--------------------------------------------------')


    # #xgboost
    # booster = xgb.XGBClassifier()

    # param = {
    # 'eta': 0.3, 
    # 'max_depth': 6,  
    # 'objective': 'binary:hinge'} 
    # steps = 15

    # model = xgb.train(param, D_train, steps)

    # y_pred_bool = model.predict(D_test)
    # accuracy = accuracy_score(y_test, y_pred_bool)
    # precision = precision_score(y_test, y_pred_bool)
    # recall = recall_score(y_test, y_pred_bool)
    # roc = roc_auc_score(y_test, y_pred_bool)
    
    
    # metrics = {'accuracy': accuracy, 'precision': precision, 'recall': recall, "roc": roc}
    # print('------------------XGB-----------------------------')
    # print(metrics)


In [10]:
model_trial(df_short)

------------------Log-----------------------------
{'accuracy': 0.9564507348938487, 'precision': 0.9827586206896551, 'recall': 0.6895161290322581, 'roc': 0.8438140745853551}
--------------------------------------------------


In [11]:
model_trial(df_medium)

------------------Log-----------------------------
{'accuracy': 0.863030303030303, 'precision': 0.8619489559164734, 'recall': 0.8741176470588236, 'roc': 0.8626838235294119}
--------------------------------------------------


In [12]:
model_trial(df_dank)

------------------Log-----------------------------
{'accuracy': 0.5757575757575758, 'precision': 0.5921375921375921, 'recall': 0.5670588235294117, 'roc': 0.5760294117647058}
--------------------------------------------------
